In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import scipy
from datetime import datetime

In [2]:
filepath = "Data/Belgium/ELIA/ods001.csv"

In [3]:
data = pd.read_csv(filepath, delimiter=";")[749:][::-1].reset_index().drop("index", axis=1)

In [4]:
data["Datetime"] = pd.to_datetime(data["Datetime"], errors='coerce', utc=True)
data["Datetime"] = data["Datetime"].dt.tz_convert("UTC")

In [5]:
data.head()

,Datetime,Resolution code,Total Load,Most recent forecast,Most recent P10,Most recent P90,Day-ahead 6PM forecast,Day-ahead 6PM P10,Day-ahead 6PM P90,Week-ahead forecast
0,2014-12-31 23:00:00+00:00,PT15M,10142.19,9496.05,8871.71,10120.38,9697.59,9076.64,10318.53,9833.37
1,2014-12-31 23:15:00+00:00,PT15M,10051.28,9329.17,8715.80,9942.53,9526.54,8916.55,10136.53,9653.31
2,2014-12-31 23:30:00+00:00,PT15M,9952.87,9174.72,8571.51,9777.92,9367.66,8767.84,9967.47,9473.22
3,2014-12-31 23:45:00+00:00,PT15M,9821.78,9025.46,8432.07,9618.85,9216.09,8625.97,9806.20,9319.45
4,2015-01-01 00:00:00+00:00,PT15M,9755.00,9222.33,8880.23,9564.44,10130.36,9777.87,10482.86,9203.25


In [6]:
data['Total Load Interpolated'] = data['Total Load'].interpolate(method='linear')

In [7]:
data["Year"] = [i.year for i in data["Datetime"]]
data["Month"] = [i.month for i in data["Datetime"]]
data["Day"] = [i.day for i in data["Datetime"]]
data["Hour"] = [i.hour for i in data["Datetime"]]
data["Minute"] = [i.minute for i in data["Datetime"]]
data["Season"] = data["Month"]//4

Lockdown dates:
- 18/03/2020 -> 04/05/2020
- 30/10/2020 -> 15/12/2020
- 20/11/2020 -> 20/01/2022

In [8]:
data["Lockdown"] = 0

In [9]:
data.loc[182784:187296, "Lockdown"] = 1
data.loc[204388:208804, "Lockdown"] = 1
data.loc[212260:247300, "Lockdown"] = 1

In [10]:
data["Difference with previous load"] = [0] + list(np.array(data["Total Load"][1:].to_list()) - np.array(data["Total Load"][:-1].to_list()))
data["Difference with previous load interpolated"] = [0] + list(np.array(data["Total Load Interpolated"][1:].to_list()) - np.array(data["Total Load Interpolated"][:-1].to_list()))

In [11]:
def get_previous_day_value(df, current_time, column):
    df.set_index("Datetime", inplace=True)  
    previous_day = current_time - pd.DateOffset(hours=1)
    previous_time = previous_day.replace(minute=45)
    if previous_time in df.index:
        return df.loc[previous_time, column]
    return None

data['Total Load Interpolated Persistence'] = data["Datetime"].apply(lambda x: get_previous_day_value(data.copy(), x, 'Total Load Interpolated'))
data['Total Load Persistence'] = data["Datetime"].apply(lambda x: get_previous_day_value(data.copy(), x, 'Total Load'))

In [12]:
data.head()

,Datetime,Resolution code,Total Load,Most recent forecast,Most recent P10,Most recent P90,Day-ahead 6PM forecast,Day-ahead 6PM P10,Day-ahead 6PM P90,Week-ahead forecast,...,Month,Day,Hour,Minute,Season,Lockdown,Difference with previous load,Difference with previous load interpolated,Total Load Interpolated Persistence,Total Load Persistence
0,2014-12-31 23:00:00+00:00,PT15M,10142.19,9496.05,8871.71,10120.38,9697.59,9076.64,10318.53,9833.37,...,12,31,23,0,3,0,0.00,0.00,NaN,NaN
1,2014-12-31 23:15:00+00:00,PT15M,10051.28,9329.17,8715.80,9942.53,9526.54,8916.55,10136.53,9653.31,...,12,31,23,15,3,0,-90.91,-90.91,NaN,NaN
2,2014-12-31 23:30:00+00:00,PT15M,9952.87,9174.72,8571.51,9777.92,9367.66,8767.84,9967.47,9473.22,...,12,31,23,30,3,0,-98.41,-98.41,NaN,NaN
3,2014-12-31 23:45:00+00:00,PT15M,9821.78,9025.46,8432.07,9618.85,9216.09,8625.97,9806.20,9319.45,...,12,31,23,45,3,0,-131.09,-131.09,NaN,NaN
4,2015-01-01 00:00:00+00:00,PT15M,9755.00,9222.33,8880.23,9564.44,10130.36,9777.87,10482.86,9203.25,...,1,1,0,0,0,0,-66.78,-66.78,9821.78,9821.78


In [13]:
data.tail()

,Datetime,Resolution code,Total Load,Most recent forecast,Most recent P10,Most recent P90,Day-ahead 6PM forecast,Day-ahead 6PM P10,Day-ahead 6PM P90,Week-ahead forecast,...,Month,Day,Hour,Minute,Season,Lockdown,Difference with previous load,Difference with previous load interpolated,Total Load Interpolated Persistence,Total Load Persistence
338410,2024-08-26 01:30:00+00:00,PT15M,6709.33,6901.54,6734.68,7068.40,6814.75,6556.32,7073.17,6860.06,...,8,26,1,30,2,0,-70.66,-70.66,6859.53,6859.53
338411,2024-08-26 01:45:00+00:00,PT15M,6841.40,6880.89,6714.53,7047.25,6794.36,6536.71,7052.01,6838.49,...,8,26,1,45,2,0,132.07,132.07,6859.53,6859.53
338412,2024-08-26 02:00:00+00:00,PT15M,6842.97,6887.46,6721.35,7053.57,6921.37,6664.11,7178.63,6948.82,...,8,26,2,0,2,0,1.57,1.57,6841.40,6841.40
338413,2024-08-26 02:15:00+00:00,PT15M,6905.57,6896.76,6730.43,7063.09,6930.71,6673.11,7188.32,6937.10,...,8,26,2,15,2,0,62.60,62.60,6841.40,6841.40
338414,2024-08-26 02:30:00+00:00,PT15M,6885.74,6910.04,6743.39,7076.69,6944.06,6685.96,7202.16,6964.77,...,8,26,2,30,2,0,-19.83,-19.83,6841.40,6841.40


In [14]:
data.to_csv("Processed_data.csv")